In [1]:
import tokenizer as tok
import pandas as pd
from multiprocessing import Process, Queue

def f(queue, ecsv, rows):
    df = pd.DataFrame()
    for index, row in ecsv.iterrows():
        if index not in rows:
            continue
        df = df.append(tok.tokenizer(index, str(row['name']) + ' ' + str(row['productdescription']), str(row['brand']), str(row['modelnumber'])))
        # if index % 1000 == 0:
        #     print('Verarbeitete Produkte:', index)
        #     df.to_csv('Daten/trainingData.csv', index = False)
    queue.put(df)

ecsv = pd.read_csv('Daten/KoepckeEigen/electronicFixed.csv',escapechar="\\",sep=",",error_bad_lines=False,warn_bad_lines=False)
ecsv.dropna(axis = 0)
ecsv = ecsv.head(200)

nP = 8
rowList = []
process = []
df = pd.DataFrame()

for i in range(nP):
    rowList.append(range(i, 2000, nP))
print(rowList)
if __name__ == '__main__':
    queue = [Queue() for i in range(nP)]
    process = [Process(target = f, args = (queue[i], ecsv, rowList[i],)) for i in range(nP)]
    for p in process:
        p.start()
    for q in queue:
        df = df.append(q.get())
    for p in process:
        p.join()

#print(dList)
#print(dList.pop(0))
#df = pd.DataFrame(dList, columns=['Id', 'satzID', 'Wort', 'Attribut'])
print(df)
#df.to_csv('Daten/trainingData.csv', index = False)

c:\Users\thilo\OneDrive\Dokumente\UniversitätLeipzig\DataScience2\BigData\tokenizer.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[b_brand.index.asi8[x]:b_brand.index.asi8[x]+len(bList)]['Attribut'] = "I-Brand"
     satzId      Wort       Attribut
0         0     Front              O
1         0  controls              O
2         0    Easily  B-Modelnumber
3         0   operate              O
4         0      your              O
..      ...       ...            ...
101       0       and              O
102       0    remove              O
103       0       for              O
104       0      easy              O
105       0  cleaning              O

[106 rows x 3 columns]
    satzId   Wort Attribut
17       0  Sensi  B-Brand
    satzI

In [2]:
import tokenizer as tok
import pandas as pd

ecsv = pd.read_csv('Daten/KoepckeEigen/electronicFixed.csv',escapechar="\\",sep=",",error_bad_lines=False,warn_bad_lines=False)
ecsv.dropna(axis = 0)
dList = []
df = pd.DataFrame()
for index, row in ecsv.iterrows():
    df = df.append(tok.tokenizer(index, str(row['name']) + ' ' + str(row['productdescription']), str(row['brand']), str(row['modelnumber'])))
    #df = df.append(tokenizer(index, str(row['name']), str(row['brand']), str(row['modelnumber'])))
    if index % 1000 == 0:
        print('Verarbeitete Produkte:', index)
        df.to_csv('Daten/trainingDataInc.csv', index = False)
#print(dList)
#print(dList.pop(0))
#df = pd.DataFrame(dList, columns=['Id', 'satzID', 'Wort', 'Attribut'])
print(df)
df.to_csv('Daten/trainingData.csv', index = False)

Verarbeitete Produkte: 0
c:\Users\thilo\OneDrive\Dokumente\UniversitätLeipzig\DataScience2\BigData\tokenizer.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[b_brand.index.asi8[x]:b_brand.index.asi8[x]+len(bList)]['Attribut'] = "I-Brand"
c:\Users\thilo\OneDrive\Dokumente\UniversitätLeipzig\DataScience2\BigData\tokenizer.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[b_modelnumber.index.asi8[x]:b_modelnumber.index.asi8[x]+len(mList)]['Attribut'] = "I-Modelnumber"
Verarbeitete Produk

KeyboardInterrupt: 

In [5]:
import pandas as pd
maxLengthList = [300, 100, 200, 250, 400, 500]
df = pd.read_csv('Daten/trainingdata.csv',escapechar="\\",sep=",",error_bad_lines=False,warn_bad_lines=False)
print(df)
for maxLength in maxLengthList:
    df = df.groupby('satzId').filter(lambda x: len(x) <= maxLength)
    print(df)
    df.to_csv('Daten/trainingData'+ str(maxLength) + '.csv', index = False)

          satzId           Wort       Attribut
0              0          Broan        B-Brand
1              0  TEN136WWBroan              O
2              0       TEN136WW  B-Modelnumber
3              0       Overview              O
4              0            The              O
...          ...            ...            ...
12432645   55340            fit              O
12432646   55340             in              O
12432647   55340        tighter              O
12432648   55340         spaces              O
12432649   55340              .              O

[12432650 rows x 3 columns]


KeyboardInterrupt: 